In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.speakernet as speakernet
import tensorflow as tf


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
config = malaya_speech.config.speakernet_featurizer_config
featurizer = malaya_speech.featurization.SpeakerNetFeaturizer(config)

In [5]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [6]:
%%time
featurizer.vectorize(y).shape

CPU times: user 12.2 ms, sys: 3.75 ms, total: 15.9 ms
Wall time: 9.56 ms


(564, 64)

In [7]:
v = tf.placeholder(tf.float32, [None, None, 64])
v_length = tf.placeholder(tf.int32, [None])

In [8]:
model = speakernet.Model(v, v_length)
model.logits


Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use `tf.keras.layers.SeparableConv1D` instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.


<tf.Tensor 'Relu_1:0' shape=(?, 512) dtype=float32>

In [9]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [12]:
padded, l = malaya_speech.padding.sequence_nd([featurizer.vectorize(y), featurizer.vectorize(y1)], 
                                              dim = 0, return_len = True)

In [13]:
sess.run(model.logits, feed_dict = {v: padded, v_length: l}).shape

(2, 512)